In [10]:
import os
import numpy as np
import cv2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.feature import hog

def load_mask_dataset(base_path, subset):
    """
    Încarcă setul de date cu masti din directorul specificat (train, val sau test)
    """
    X = []  # va conține descriptorii HOG
    y = []  # va conține etichetele 
    original_images = []  # pentru vizualizare

   
    expected_pieces = ['mask', 'non mask']

    subset_path = os.path.join(base_path, subset)
    print(f"Încărcare set de date pentru subsetul: {subset}...")

    # Parcurgem fiecare subdirector
    for piece in os.listdir(subset_path):
        if piece.lower() in expected_pieces:
            piece_path = os.path.join(subset_path, piece)
            print(f"Procesare {piece}...")

            # Încărcăm fiecare imagine din subdirector
            for img_name in os.listdir(piece_path):
                img_path = os.path.join(piece_path, img_name)
                if img_path.endswith(('.jpg', '.png', '.jpeg')):
                    img = cv2.imread(img_path)
                    if img is None:
                        print(f"Imaginea {img_path} nu a putut fi încărcată.")
                        continue

                    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    img_gray = cv2.resize(img_gray, (64, 64))  # Redimensionare
                    original_images.append(cv2.resize(img, (64, 64)))

                    # Extragere caracteristici HOG
                    features, _ = hog(img_gray, orientations=9, pixels_per_cell=(8, 8),
                                      cells_per_block=(2, 2), visualize=True, channel_axis=None)
                    X.append(features)
                    y.append(piece.lower())  # Normalizare etichete

    if len(X) == 0 or len(y) == 0:
        print(f"EROARE: Nu au fost găsite date valide în {subset_path}. Verifică structura setului de date.")

    X = np.array(X)
    y = np.array(y)
    original_images = np.array(original_images)

    return X, y, original_images

def display_test_results(test_images, test_labels, knn_predictions, nb_predictions):

    """
    Afișează imaginile de test cu etichetele reale și prezise de KNN și Naive Bayes.
    """
    for idx, (img, actual, knn_pred, nb_pred) in enumerate(zip(test_images, test_labels, knn_predictions, nb_predictions)):
        plt.figure(figsize=(3, 3))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        title = (f"Imagine {idx + 1}\nReal: {actual}\n"
                 f"KNN: {knn_pred}\nNaive Bayes: {nb_pred}")
        plt.title(title, fontsize=10)
        print(f"Imagine {idx + 1}: Real - {actual}, KNN - {knn_pred}, Naive Bayes - {nb_pred}")
        plt.show()


def plot_confusion_matrix(y_true, y_pred, class_names, title):
    """
    Afișează matricea de confuzie
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.xlabel("Etichete prezise")
    plt.ylabel("Etichete reale")
    plt.show()

def train_and_evaluate_models(X_train, X_val, X_test, y_train, y_val, y_test):
    """
    Antrenează și evaluează modelele KNN și Naive Bayes
    """
    # Verificăm dimensiunile datelor
    print(f"Dimensiuni X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Dimensiuni X_val: {X_val.shape}, y_val: {y_val.shape}")
    print(f"Dimensiuni X_test: {X_test.shape}, y_test: {y_test.shape}")

    # Inițializare modele
    knn = KNeighborsClassifier(n_neighbors=)
    nb = GaussianNB()

    # Antrenare modele
    print("\nAntrenare modele...")
    knn.fit(X_train, y_train)
    nb.fit(X_train, y_train)

    # Evaluare pe setul de validare
    print("\nKNN (Validare):")
    y_pred_knn_val = knn.predict(X_val)
    print(classification_report(y_val, y_pred_knn_val))

    print("\nNaive Bayes (Validare):")
    y_pred_nb_val = nb.predict(X_val)
    print(classification_report(y_val, y_pred_nb_val))

    # Evaluare finală pe setul de test
    print("\nKNN (Test):")
    y_pred_knn_test = knn.predict(X_test)
    print(classification_report(y_test, y_pred_knn_test))

    print("\nNaive Bayes (Test):")
    y_pred_nb_test = nb.predict(X_test)
    print(classification_report(y_test, y_pred_nb_test))

    return knn, nb, y_pred_knn_test, y_pred_nb_test

# Exemplu de utilizare
if __name__ == "__main__":
    dataset_path = "/content/drive/MyDrive/New_Masks_Dataset"

    # Încărcăm datele
    X_train, y_train, _ = load_mask_dataset(dataset_path, "Antrenare")
    X_val, y_val, _ = load_mask_dataset(dataset_path, "Validare")
    X_test, y_test, test_images = load_mask_dataset(dataset_path, "Testare")

    print(f"\nTotal imagini încărcate:")
    print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

    # Verificăm distribuția claselor
    print("\nDistribuția claselor:")
    for set_name, y_set in [("Antrenare", y_train), ("Validare", y_val), ("Testare", y_test)]:
        print(f"\n{set_name}:")
        unique, counts = np.unique(y_set, return_counts=True)
        for piece, count in zip(unique, counts):
            print(f"{piece}: {count} imagini")

    # Antrenăm și evaluăm modelele
    knn_model, nb_model, y_pred_test_knn, y_pred_test_nb = train_and_evaluate_models(X_train, X_val, X_test, y_train, y_val, y_test)

    # Afișăm imaginile de test și rezultatele predicțiilor
    print("\nAfișare imagini de test cu predicții...")
    display_test_results(test_images, y_test, y_pred_test_knn, y_pred_test_nb)

    # Afișăm matricea de confuzie pentru KNN și Naive Bayes
    print("\nMatrice de Confuzie pentru KNN (Test):")
    class_names = np.unique(y_test)
    plot_confusion_matrix(y_test, y_pred_test_knn, class_names, "Matrice de Confuzie pentru KNN")

    print("\nMatrice de Confuzie pentru Naive Bayes (Test):")
    plot_confusion_matrix(y_test, y_pred_test_nb, class_names, "Matrice de Confuzie pentru Naive Bayes")


Output hidden; open in https://colab.research.google.com to view.